In [12]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#
#
#                                                  project 2 - Objecto em AR
#
#
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------


import cv2 
import cv2.aruco as aruco 
import numpy as np
#import time 
import trimesh
import pyrender
from PIL import Image
import trimesh.visual
import os
os.environ["PYOPENGL_PLATAFORM"] = "egl"

self = 0 
# verify disponibilidade 

if not hasattr(cv2, 'aruco'):
    raise ImportError("Not found cv2.aruco")

print("Version openvc: ", cv2.__version__)

class Project2:

    def __init__(self):
        data = np.load("calibration_data.npz")
        self.aruco_dir = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_50)
        self.mtx = data['mtx']
        self.dist = data['dist']      # correscopende a matriz intrínseca M = K[R|t]
        self.marker_length = 0.03
        # obj vitor
        #self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\guy.obj"
        #self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\source\pou\pou.obj"
        #self.texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\textures\Sin_título.png", cv2.IMREAD_COLOR)
        # turtle
        self.objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_v1_l2.obj"
        img_bgr = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_diffuse.jpg")
        img_bgr = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        self.texture_img = Image.fromarray(img_bgr)
        #self.texture_img = Image.open(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_diffuse.jpg")
        self.renderer = pyrender.OffscreenRenderer(viewport_width=640, viewport_height=480)
        print("Renderer context:", self.renderer._platform)
        print(f"mtx: {self.mtx}")
        print(f"dist: {self.dist}")

   
    def gerar_Aruco(self):
        # gerando marcadores Aruco 

        # marcadores testes 5
        num_markers = 5 

        # tamanho da imagem do marcador (em pixels)
        marker_size = 200

        
        for marker_id in range(num_markers):
            # criar imagem do marcador 
            marker_img = cv2.aruco.generateImageMarker(self.aruco_dir, marker_id, marker_size)

            # salvando o tempplate 
            filename = f'marker_{marker_id}.png'
            cv2.imwrite(filename, marker_img)

            print(f"well done save: {filename}")

    
    def desenhar_objecto(self, frame, rvec, tvec, corners):
        size = self.marker_length
         
        # desenhando os pontos para forma geometrica
        p1 = (corners[0][0][0][0], corners[0][0][0][1])
        p2 = (corners[0][0][1][0], corners[0][0][1][1])
        p3 = (corners[0][0][2][0], corners[0][0][2][1])
        p4 = (corners[0][0][3][0], corners[0][0][3][1])
        # definindo a reta do desenho 
        b1, b2 = p1[0], p1[1]
        b3, b4 = p2[0], p2[1]
        b5, b6 = p3[0], p3[1]
        b7, b8 = p4[0], p4[1]

        # piramede 
        # parte inferior
        # calculando os pontos centrais entre (b1-b5) e (b3-b7)
        cv2.line(frame, (int(b1), int(b2)), (int(b3), int(b4)), (0, 255, 255), 3)
        cv2.line(frame, (int(b5), int(b6)), (int(b7), int(b8)), (0, 255, 255), 3)
        cv2.line(frame, (int(b1), int(b2)), (int(b7), int(b8)), (0, 255, 255), 3)
        cv2.line(frame, (int(b3), int(b4)), (int(b5), int(b6)), (0, 255, 255), 3)

        # corners 
        x1, y1 = (b1 + b5) // 2, (b2 + b6) // 2
        x2, y2 =  (b3 + b7) // 2, (b4 + b8) // 2
        # criando o "topo" da pirâmide deslocado para cima (y - 200)
        cv2.line(frame, (int(b1), int(b2)), (int(x1), int(y1 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b5), int(b6)), (int(x1), int(y1 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b3), int(b4)), (int(x1), int(y2 - 200)), (0, 255, 255), 3)
        cv2.line(frame, (int(b7), int(b8)), (int(x1), int(y2 - 200)), (0, 255, 255), 3)

        """# base do objecto 
        base = 0   # delimitação bordas
        height = size * 0.5
        # desenho parametros
        axis = np.float32([
            [0, 0, 0 ],
            [size, 0, 0],
            [size, size, 0],
            [0, size, 0],
            [size / 2, size / 2, height]
        ])

        imgpts, _ = cv2.projectPoints(axis, rvec, tvec, self.mtx, self.dist)
        imgpts = np.int32(imgpts).reshape(-1, 2)

        # base do objecto
        #cv2.drawContours(frame, [imgpts[:4]], -1, (0, 255, 0), 2)
        base_pts = np.array([imgpts[0], imgpts[1], imgpts[2], imgpts[3]])
        cv2.polylines(frame, [base_pts], isClosed=True, color=(0, 255, 255), thickness=2)

        # ligando forma geometrica
        for i in range(4):
            cv2.line(frame, tuple(imgpts[i]), tuple(imgpts[4]), (0, 255, 255), 3)"""

        
        return frame

    def carregar_obj(self, pathobj): 
        # lendo ficheiro .obj 
        vertices = []
        faces = []

        with open(pathobj, 'r') as f:
            for line in f:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = list(map(float, parts[1:4]))
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]   # 
                    faces.append(face)

        return np.array(vertices, dtype=np.float32), faces

    def tratar_obj(face_line):
        face = []
        for vert in face_line.strip().split()[1:]:
            parts = vert.slipt('/')
            v_idx = int(parts[0]) - 1           # indexado a partir de 1
            face.append(v_idx)
        return face


    def carregar_obj_textura(self, pathobj): 
        # lendo ficheiro .obj 
        vertices = []
        faces = []
        texcoords = []

        with open(pathobj, 'r') as f:
            for line in f:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = list(map(float, parts[1:4]))
                    vertices.append(vertex)
                elif line.startswith('vt '):
                    parts = line.strip().split()
                    texcoord = list(map(float, parts[1:3]))
                    texcoords.append(texcoord)
                elif line.startswith('f '):
                    parts = line.strip().split()[1:]
                    raw_face = []
                    for p in parts:
                        if '/' in p: 
                            indices = p.split('/')
                            v_idx = int(indices[0]) - 1
                            vt_idx = int(indices[1]) - 1 if len(indices) > 1 and indices[1] != '' else 0
                            #v_idx, vt_idx = map(lambda x: int(x) - 1, p.split('/')[:2])
                            raw_face.append((v_idx, vt_idx))
                        
                    # verifica se a face é um quadrilatero
                    if len(raw_face) == 4:
                        # converte quadrilateros
                        f1 = [raw_face[0], raw_face[1], raw_face[2]]
                        f2 = [raw_face[0], raw_face[2], raw_face[3]]
                        faces.append(f1)
                        faces.append(f2)
                        print(f"Face dividida em 2 triangulos.")
                    elif len(raw_face) == 3:
                        faces.append(raw_face)
                    else:
                        print(f"Face com {len(raw_face)} vértices ignorada (não é triângulo nem quadrado).")

        print(f"Total de vértices carregadas: {len(vertices)}")
        print(f"Total de coordedanas de textura carregadas: {len(texcoords)}")
        print(f"Total de faces triangulares: {len(faces)}")

                                            
                    
        return np.array(vertices, dtype=np.float32), np.array(texcoords, dtype=np.float32), faces
    
    def mesh_textura(self, vertices, texcoords, faces):
        # indices de arrays 
        new_vert = []
        new_cords = []
        new_faces = []

        for face in faces:
            face_indice = []
            print(f"Face actual: {face}")
            for vi, ti in face:
                new_vert.append(vertices[vi])
                new_cords.append(texcoords[ti])
                face_indice.append(len(new_vert) - 1)
            new_faces.append(face_indice)

        new_faces = np.array(new_faces, dtype=np.int32)
        new_cords = np.array(new_cords, dtype=np.float32)
        new_vert = np.array(new_vert, dtype=np.float32)

               
        print("new vertices: ", new_vert)
        print("new faces: ", new_faces)
        #vert_idx = np.array(vert_idx, dtype=np.int32).reshape((-1, 3))
        #tex_idx = np.array(tex_idx, dtype=np.int32).reshape((-1, 3))
        
        # reeordenando o tex_idx 
        #uv_coords = np.array(texcoords, dtype=np.float32)[tex_idx.flatten()]
        print(new_faces)
        

        """# carregando a textura 
        if isinstance(self.texture_img, np.ndarray):
            textura = Image.fromarray(cv2.cvtColor(self.texture_img, cv2.COLOR_BGR2RGB)).convert('RGB')
        else:
            textura = Image.open(self.texture_img).convert('RGB')"""
        textura = self.texture_img.convert('RGBA')
        

        # Atribui coordenadas UV ao mesh
        #trimesh_visual = trimesh.visual.TextureVisuals(uv=texcoords, image=textura)
        #texcoords = np.array(texcoords, dtype=np.float32)
        print(f"Shape das texcoords:{texcoords.shape}")
        """if texcoords.ndim == 1:
            texcoords = texcoords.reshape(-1, 2)
        elif texcoords.shape[1] > 2:
            texcoords = texcoords[:, :2]"""

        print(f"New texcoords: ",new_cords.shape)
        tex = pyrender.Texture(source=textura, source_channels='RGB')
        print(tex)
        
        # mesh da textura a ser carregada 
        visual = trimesh.visual.TextureVisuals(uv=new_cords, image=textura)

        # trimesh parameters:
        mesh = trimesh.Trimesh(vertices=new_vert, faces=new_faces, visual=visual, validate=True, process=False)

        print("mesh", mesh)
        print(mesh.visual.uv.shape)
        assert mesh.visual.uv.shape[0] == mesh.vertices.shape[0], "Número de UVs deve coincidir com número de vértices"

        print("Tipo de textura:", type(textura))
        print("Imagem RGB:", textura.mode)
        print("Size:", textura.size)


        # adcicionando a  textura 
        material = pyrender.MetallicRoughnessMaterial(
                baseColorTexture=pyrender.Texture(source=textura, source_channels='RGB'),
                metallicFactor=0.0, 
                roughnessFactor=1.0
            )

        # constroi mesh para o pyrender 
        render_mesh = pyrender.Mesh.from_trimesh(mesh, material=material, smooth=True)

        print("render mesh final", render_mesh)
        return render_mesh

    def renderizacao_obj(self, frame, rvecs, tvecs):
        # carregando o modelo 
        vertices, texcoords, faces = self.carregar_obj_textura(self.objpath)
        vertices = self.normalizar_obj(vertices, scale=0.05)
        mesh = self.mesh_textura(vertices, texcoords, faces)


        # cria cena
        cena = pyrender.Scene(bg_color=[0, 0, 0, 0], ambient_light=[0.1, 0.1, 0.1])
        #cena.add(mesh)

        # configurando a luz
        luz = pyrender.DirectionalLight(color=np.ones(3), intensity=2.0)
        cena.add(luz)

        # criando a matriz de projeção da pose a partit dos vtores de translação e rotação
        rot, _ = cv2.Rodrigues(rvecs)
        pose = np.eye(4)
        pose[:3, :3] = rot 
        pose[:3, 3] = tvecs.flatten()
        cena.add(mesh, pose=pose)

        # camera projeteda a partir da intrínseca OpenCV
        fx = self.mtx[0, 0]
        fy = self.mtx[1, 1]
        cx = self.mtx[0, 2]
        cy = self.mtx[1, 2]
        height, width = frame.shape[:2]

        cam = pyrender.IntrinsicsCamera(fx=fx, fy=fy, cx=cx, cy=cy)
        cena.add(cam, pose=np.eye(4))

        # renderizando a imagem 
        color, depth = self.renderer.render(cena)
        pyrender.viewer(cena, use_raymond_lighting=True)

        # convertendo open CV BGR 
        color_bgr = cv2.cvtColor(color, cv2.COLOR_RGBA2BGRA)

        # faz faz um overlay do objecto 3D na imagem da camera
        mask = color_bgr[:, :, 3] > 0
        frame[mask] = color_bgr[mask][:, :3]

        return frame

    def exibir_projecao(self, render_mesh):
        cena = pyrender.Scene()
        cena.add(render_mesh)

        # controlo luz e camera
        luz = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
        cena.add(luz)

        cam = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        cena.add(cam, pose=np.eye(4))

        viewer = pyrender.viewer(cena, use_raymond_lighting=True, run_in_thread=False)
  
    def objecto_texturizado(self, frame, vertices, texcoords, faces, rvec, tvec):
        imgpts, _ = cv2.projectPoints(vertices, rvec, tvec, self.mtx, self.dist)
        imgpts = imgpts.reshape(-1, 2)

        # valida se textura esta carregada 
        if self.texture_img is None:
            print("A Textura da imagem não esta sendo carregada !")
            return frame

        h, w, _ = self.texture_img.shape


        for face in faces:
            print(f"face projetada: {face}")
            if len(face) != 3:
                print(f"Ignorando face com {len(face)} véritices")
                continue
                                # apenas triângulos suportados neste exemplo
            try:
                pts_3d = np.array([imgpts[v_idx] for v_idx, _ in face], dtype=np.float32)
                pts_uv = np.array(
                    [[texcoords[vt_idx][0] * w,
                    (1 - texcoords[vt_idx][1]) * h] 
                    for _, vt_idx in face], 
                    dtype=np.float32
                    )
                print(pts_3d, "teste")
                # verificar se tem os pontos exatos para projeção 
                if pts_3d.shape[0] != 3 or pts_uv.shape[0] !=3:
                    print(f"Erro: Face não tem 3 pontos necessarios, ignorando.....")
                    continue
                    
                # matrix de tranformação para o frame
                #if len(pts_3d) == 3 and len(pts_uv) == 3:
                M = cv2.getAffineTransform(pts_uv, pts_3d)

                # aplicar a transformação a textura
                triangle = cv2.warpAffine(self.texture_img, M,
                                           (frame.shape[1], frame.shape[0]), 
                                           flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)
                    
                # Máscara do triângulo para aplicar a textura apenas no triangulo
                mask = np.zeros_like(frame)
                cv2.fillConvexPoly(mask, pts_3d.astype(np.int32), (255, 255, 255))

                frame = np.where(mask == 0, frame, triangle)
                #triangle_masked = cv2.bitwise_and(triangle, mask)
                #frame_masked = cv2.bitwise_and(frame, cv2.bitwise_not(mask))
                #frame = cv2.add(frame_masked, triangle_masked)    
            except Exception as e:
                print(f"Erro ao processar face:{e}")
                continue

        return frame

    def objecto_3d(self, frame, vertices, faces, rvec, tvec):

        #pou 
        #texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pou-obj\textures\Sin_título.png")
        # turtle 
        #texture_img = cv2.imread(r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_diffuse.jpg")

        # projeta os vertices 
        imgpts, _ = cv2.projectPoints(vertices, rvec, tvec, self.mtx, self.dist)
        imgpts = imgpts.reshape(-1, 2).astype(int)

        # Remover valores NaN
        if np.isnan(imgpts).any():
            print("Vertices inválidos detectados.")
            return frame

        for face in faces:
            pts = np.array([imgpts[idx] for idx in face], dtype=np.int32) # array de array ## buscando os pontos e desenhando 
            #print(pts)
            # adcionando pontos de projetção 
            cv2.polylines(frame, [pts], isClosed=True, color=(0, 255, 255), thickness=1)
            #cv2.fillPoly(frame, [pts], color=(0, 255, 100))

            # adcionado textura projetada 
            #color = cv2.mean(texture_img)[0:3]
            #color = tuple(map(int, color))
            #cv2.fillConvexPoly(frame, pts, color=color)
        
        return frame

    def normalizar_obj(self, vertices, scale):
        # centralizando 
        centro = np.mean(vertices, axis=0)
        #centro[2]=centro[2] -10
        print(f"centro: {centro}")
        vertices -= centro            # alinhando no centro da marker , retirando a translação

        # equadrando no espaço 
        max_dim = np.max(np.linalg.norm(vertices, axis=1))
        vertices = (vertices / max_dim) * scale

        return vertices


    def main(self):
        # open camera 
        cap = cv2.VideoCapture(0)
        # define resolution
        #cap.set(3, 1280)
        #cap.set(4, 720)
        # carregando paramêtros 
        parameters = aruco.DetectorParameters()
        # deteção parametros 
        detect = aruco.ArucoDetector(self.aruco_dir, parameters)

        # carregando a camera 
        while True:
            # carregando a camera 
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if not ret:
                break

            # deteção marcadores 
            corners, ids, _ = detect.detectMarkers(gray)
            

            # marcadores selecionados 
            
            if ids is not None and len(ids) > 0:

                for i in range(len(ids)):
                    # estimar a pose da camera 
                    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners[i], self.marker_length, self.mtx, self.dist)

                    # tratando possiveis erros
                    (rvecs - tvecs).any()

                    # definindo ID 
                    marker_id = ids[i][0]
                    
                    # delimitando bordas
                    cv2.aruco.drawDetectedMarkers(frame, corners)

                    # projectando eixos 
                    cv2.drawFrameAxes(frame, self.mtx, self.dist, rvecs, tvecs, self.marker_length)

                    # coordenada x
                    X = (corners[i][0][0][0] + corners[i][0][1][0] + corners[i][0][2][0] + corners[i][0][3][0]) / 4  # dividir cantos do x definir o ponto do meio 

                    # coordenadas Y
                    Y = (corners[i][0][0][1] + corners[i][0][1][1] + corners[i][0][2][1] + corners[i][0][3][1]) / 4  # dividir cantos do y , definindo o ponto do meio


                    print(X)
                    print(Y)
                    #
                    # exibir id
                    cv2.putText(frame, "ID: " + str(marker_id), (int(X), int(Y)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
                    
                    # desenhando a objectos simples (pirâmede) 
                    #frame = self.desenhar_objecto(frame, rvecs, tvecs, corners)

                    # carregando objecto 
                    #self.vertices , self.faces = self.carregar_obj(self.objpath)
                    #print(f"Vertices: {len(self.vertices)}")
                    #print(f"faces: {self.faces}")

                    # objecto com texturas 
                    #self.vertices , self.texcoords, self.faces  = self.carregar_obj_textura(self.objpath)


                    # normalizndo objecto 
                    #self.vertices = self.normalizar_obj(self.vertices, scale=0.05)

                    # definidno a mesh do obj 
                    #self.mesh = self.mesh_textura(self.vertices, self.texcoords, self.faces)
                    """ 
                    # desenhando objecto 3d 
                    frame = self.objecto_3d( 
                        frame, 
                        self.vertices, 
                        self.faces,
                        rvecs,
                        tvecs
                    )"""
                    """# desenhando objecto 3d com textura 
                    frame = self.objecto_texturizado( 
                        frame, 
                        self.vertices, 
                        self.texcoords,
                        self.faces,
                        rvecs,
                        tvecs
                    )"""
                    frame = self.renderizacao_obj(frame, rvecs[i], tvecs[i])

                    # mostrar vetor de translação Z (profundidade)
                    z = tvecs[i][0][2]  # acesso ao valor Z
                    print(f"ID {marker_id} - Distância Z: {z:.2f}m")

            cv2.imshow('ARUCO Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


         


if __name__ == '__main__':

    exec = Project2()
    exec.main()
    #exec.gerar_Aruco()







Version openvc:  4.11.0
Renderer context: <pyrender.platforms.pyglet_platform.PygletPlatform object at 0x000001B7D68F3530>
mtx: [[572.83075191   0.         350.28398836]
 [  0.         573.48244374 268.31448992]
 [  0.           0.           1.        ]]
dist: [[-0.10965985  0.15942934  0.01340477 -0.00419671 -0.56787964]]
383.75
402.75
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face divid

ArgumentError: ("argument 2: TypeError: No array-type handler for type _ctypes.type (value: <cparam 'P' (0x000001B7D9884518)>) registered", (1, <cparam 'P' (0x000001B7D9884518)>))

In [3]:
objpath = r"C:\Users\marce\OneDrive\Desktop\Masters_degree\2_Semestre\VARM\project_2\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\pearlturtle_v1_L2.123c8b254914-0c70-4cc6-bc03-8d58f36d90c5\13103_pearlturtle_v1_l2.obj"


def carregar_obj_textura(pathobj): 
        # lendo ficheiro .obj 
        vertices = []
        faces = []
        texcoords = []

        with open(pathobj, 'r') as f:
            for line in f:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = list(map(float, parts[1:4]))
                    vertices.append(vertex)
                elif line.startswith('vt '):
                    parts = line.strip().split()
                    texcoord = list(map(float, parts[1:3]))
                    texcoords.append(texcoord)
                elif line.startswith('f '):
                    parts = line.strip().split()[1:]
                    raw_face = []
                    for p in parts:
                        if '/' in p: 
                            indices = p.split('/')
                            v_idx = int(indices[0]) - 1
                            vt_idx = int(indices[1]) - 1 if len(indices) > 1 and indices[1] != '' else 0
                            #v_idx, vt_idx = map(lambda x: int(x) - 1, p.split('/')[:2])
                            raw_face.append((v_idx, vt_idx))
                        
                    # verifica se a face é um quadrilatero
                    if len(raw_face) == 4:
                        # converte quadrilateros
                        f1 = [raw_face[0], raw_face[1], raw_face[2]]
                        f2 = [raw_face[0], raw_face[2], raw_face[3]]
                        faces.append(f1)
                        faces.append(f2)
                        print(f"Face dividida em 2 triangulos.")
                    elif len(raw_face) == 3:
                        faces.append(raw_face)
                    else:
                        print(f"Face com {len(raw_face)} vértices ignorada (não é triângulo nem quadrado).")

        print(f"Total de vértices carregadas: {len(vertices)}")
        print(f"Total de coordedanas de textura carregadas: {len(texcoords)}")
        print(f"Total de faces triangulares: {len(faces)}")

                                            
                    
        return np.array(vertices, dtype=np.float32), np.array(texcoords, dtype=np.float32), faces


def tratar_obj(face_line):
        face = []
        for vert in face_line.strip().split()[1:]:
            parts = vert.slipt('/')
            v_idx = int(parts[0]) - 1           # indexado a partir de 1
            face.append(v_idx)
        return face


carregar_obj_textura(objpath)

Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face dividida em 2 triangulos.
Face div

(array([[ 0.6002, -2.9692, -1.1045],
        [ 0.5423, -2.4673, -1.2633],
        [ 0.831 , -2.5022, -1.1805],
        ...,
        [-1.3533,  7.6757,  0.6199],
        [-1.7861,  7.4384,  0.5371],
        [ 1.713 ,  7.1705,  0.9052]], dtype=float32),
 array([[0.7795, 0.9289],
        [0.7774, 0.9078],
        [0.789 , 0.9101],
        ...,
        [0.2629, 0.6018],
        [0.2386, 0.5927],
        [0.4227, 0.5735]], dtype=float32),
 [[(0, 0), (1, 1), (2, 2)],
  [(0, 0), (2, 2), (3, 3)],
  [(1, 1), (4, 4), (5, 5)],
  [(1, 1), (5, 5), (2, 2)],
  [(6, 6), (7, 7), (2, 2)],
  [(6, 6), (2, 2), (5, 5)],
  [(7, 7), (8, 8), (3, 3)],
  [(7, 7), (3, 3), (2, 2)],
  [(9, 9), (10, 10), (11, 11)],
  [(9, 9), (11, 11), (12, 12)],
  [(10, 10), (13, 13), (14, 14)],
  [(10, 10), (14, 14), (11, 11)],
  [(15, 15), (16, 16), (11, 11)],
  [(15, 15), (11, 11), (14, 14)],
  [(16, 16), (17, 17), (12, 12)],
  [(16, 16), (12, 12), (11, 11)],
  [(18, 18), (19, 19), (20, 20)],
  [(18, 18), (20, 20), (21, 21)],
  